In [ ]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np 
import mysql.connector

### Outils/Function

In [ ]:

config = {
  'user': 'root',
  'password': 'root',
  'host': 'localhost',
  "port": 3306,
  'database': 'sdd4',
  'raise_on_warnings': True
}

bdd = mysql.connector.connect(**config)
FIGSIZE = (10,6)

In [ ]:
def import_financialbehavior_table(bdd)->pd.DataFrame:
    cursor = bdd.cursor()
    cursor.execute("SELECT * FROM financial_behavior")
    columns = [col[0] for col in cursor.description]
    df = pd.DataFrame(cursor.fetchall(),columns = columns)
    df = df.set_index("idTB")

    return df 

def import_tweets_table(bdd) -> pd.DataFrame: 
    cursor = bdd.cursor()
    cursor.execute("SELECT * FROM tweets")
    columns = [col[0] for col in cursor.description]
    df = pd.DataFrame(cursor.fetchall(),columns = columns)
    df["observation_date"] = pd.to_datetime(df["observation_date"])
    df = df.set_index("idT")

    return df 

def import_bigmacindex_table(bdd) -> pd.DataFrame: 
    cursor = bdd.cursor()
    cursor.execute("SELECT * FROM bigmacindex")
    columns = [col[0] for col in cursor.description]
    df = pd.DataFrame(cursor.fetchall(),columns = columns)
    df["observation_date"] = pd.to_datetime(df["observation_date"])
    for columns in df.columns: 
        if columns != "observation_date": 
            df[columns] = df[columns].astype(float)
    df = df.set_index("idBM")

    return df 

def import_bitcoinperformance_table(bdd) ->pd.DataFrame: 
    cursor = bdd.cursor()
    cursor.execute("SELECT * FROM bitcoinperformance")
    columns = [col[0] for col in cursor.description]
    df = pd.DataFrame(cursor.fetchall(),columns = columns)
    df["observation_date"] = pd.to_datetime(df["observation_date"])
    for columns in df.columns: 
        if columns != "observation_date": 
            df[columns] = df[columns].astype(float)
    df = df.set_index("idB")

    return df 

def import_sp500_table(bdd)->pd.DataFrame:
    cursor = bdd.cursor()
    cursor.execute("SELECT * FROM standardandpoor")
    columns = [col[0] for col in cursor.description]
    df = pd.DataFrame(cursor.fetchall(),columns = columns)
    df["observation_date"] = pd.to_datetime(df["observation_date"])
    df = df.set_index("idSP")

    return df

def import_inflation_table(bdd) ->pd.DataFrame:

    cursor = bdd.cursor()
    cursor = bdd.cursor()
    cursor.execute("SELECT * FROM inflation")
    columns = [col[0] for col in cursor.description]
    df = pd.DataFrame(cursor.fetchall(),columns=columns)
    df["observation_date"] = pd.to_datetime(df["observation_date"])
    df["CPI"] = df["CPI"].astype(float)
    df = df.set_index("idI")

    return df

def import_joinTable(bdd,*args:pd.DataFrame): 
    df = import_sp500_table(bdd)
    for table in args: 
        df = df.merge(table,on = "observation_date",how = "inner")

    
    return df

In [ ]:

def window_data(df:pd.DataFrame):
    x = []
    y = [] 
    for i in range(len(df)-1):
        x.append(df.iloc[i])
        y.append(df.iloc[i + 1])

    return x,y

def plot_corr(df:pd.DataFrame,figsize:tuple = FIGSIZE,title: str = None) -> None: 
    plt.figure(figsize=figsize)
    sns.heatmap(df.corr(),annot=True, fmt=".2f", cmap="coolwarm")
    if title:
        plt.title(title)
    plt.xticks(fontsize=8)  
    plt.yticks(fontsize=8)

    plt.show()

def plot_linear(df:pd.DataFrame,title:str = None,figsize:tuple = FIGSIZE) -> None:
    x,y = window_data(df)
    plt.figure(figsize=figsize)
    plt.plot(x,y,"o")
    if title:
        plt.title(title)
    plt.xlabel(f"{df.name} Times: t")
    plt.ylabel(f"{df.name} Times: t+1")

    plt.show()

def plot_datetime(df:pd.DataFrame,y_index_name: str,date_index_name: str = "observation_date",title: str = None,figsize:tuple = FIGSIZE) -> None:
    x = pd.to_datetime(df[date_index_name])
    y = df[y_index_name]
    plt.figure(figsize=figsize)
    plt.plot(x,y)
    plt.xlabel(y_index_name)
    plt.ylabel(date_index_name)
    if title: 
        plt.title(title)

    plt.show()

def plot_boxplot(df:pd.DataFrame,x_index_name:str,figsize:tuple = FIGSIZE)->None:
    plt.figure(figsize=figsize)
    plt.boxplot(df[x_index_name])
    plt.title(f"Boxplot de {x_index_name}")
    plt.show()
    
def plot_histplot(df:pd.DataFrame,x_index_name,figsize: tuple = FIGSIZE, y_label: str = "nombre d'observation",bins: int = 50)->None:
    plt.figure(figsize=figsize)
    sns.histplot(df[x_index_name],bins= bins)
    plt.title(f"Distribution de {x_index_name}")
    plt.show()

### 1. Analyse descriptive 

### 1.1Inflation 

In [ ]:
inflation = import_inflation_table(bdd)
inflation

In [ ]:
inflation.iloc[:,1::].describe().round(2)

In [ ]:
plot_boxplot(inflation,"CPI")

In [ ]:
plot_datetime(inflation,"CPI",title="Évolution de l'indice des prix (CPI)")

In [ ]:

plot_histplot(inflation,"CPI")

In [ ]:
plot_linear(inflation["CPI"],"Plot lineaire du CPI")

### 1.2. Table StandarAndPoor

In [ ]:
df = import_sp500_table(bdd)
df


In [ ]:
df.iloc[:,1::].describe().round(2)

In [ ]:
plot_linear(df["realEarnings"],title= "Plot lineaire de Real Earnings")



In [ ]:
plot_corr(df.iloc[:, 1:],title="Diagrame de correlation SP500")

In [ ]:
plot_datetime(df,"realEarnings")

In [ ]:
plot_boxplot(df,"realEarnings")

### 1.3. Bitcoin performance

In [ ]:
df = import_bitcoinperformance_table(bdd)
df

In [ ]:
df.iloc[:,1::].describe().round(2)

### 1.4. BigMac Index 


In [ ]:
df = import_bigmacindex_table(bdd)
df

In [ ]:
df.iloc[:,1::].describe().round(2)

### 1.5. Tweets 

In [ ]:
df = import_tweets_table(bdd)
df 

### 1.6. Financial Behavior 

In [ ]:
df = import_financialbehavior_table(bdd)
df

### 2. Analyse Bivariée

In [ ]:
df = import_joinTable(bdd,import_bitcoinperformance_table(bdd))
df

In [ ]:
np.corrcoef(df["earnings"],df["Close"])

In [ ]:
df = import_joinTable(bdd,import_inflation_table(bdd))
df

In [ ]:
np.corrcoef(df["earnings"],df["CPI"])

In [ ]:
df = import_joinTable(bdd,import_bigmacindex_table(bdd))
df

In [ ]:
np.corrcoef(df["earnings"],df["dollar_price"])